<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Insect Model

_by Jeremy Freeman (CarbonPlan), October 26, 2020_

This notebook demos insect modelling.


In [ ]:
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load, fit, utils, prepare

In [ ]:
setup.plotting(remote=False)

Load the raw FIA data grouped by repeated inventories


In [ ]:
df = load.fia(store="local", states="conus", group_repeats=True)
df = load.terraclim(
    store="local",
    tlim=(int(df["year_0"].min()), 2020),
    data_vars=["ppt", "tavg", "pdsi", "cwd", "pet", "vpd"],
    data_aggs=["sum", "mean", "mean", "sum", "mean", "mean"],
    df=df,
    group_repeats=True,
)

Prepare data for model fitting


In [ ]:
x, y, meta = prepare.insects(df)

In [ ]:
x_z, x_mean, x_std = utils.zscore_2d(x)

Fit models


In [ ]:
pf = meta.copy()
pf["observed"] = y
pf["predicted"] = np.NaN

models = {}
for code in pf["type_code"].unique():
    inds = pf["type_code"] == code
    if (y[inds].sum() > 1) & (y[inds].sum() > 1):
        model = fit.hurdle(x=x_z[inds], y=y[inds])
        models[code] = model
        pf.loc[inds, "predicted"] = model.predict(x=x_z[inds])

pf = pf[pf["predicted"] < 1]

Compute simple score metrics (on all data and just on the values where fraction
insects was positive)


In [ ]:
pf[["observed", "predicted"]].corr().iloc[0, 1] ** 2

In [ ]:
pf[pf["observed"] > 0][["observed", "predicted"]].corr().iloc[0, 1] ** 2

Plot observed vs predicted


In [ ]:
plot.xy(
    x=pf["observed"],
    y=pf["predicted"],
    xlabel="measured fraction insects",
    ylabel="predicted fraction insects",
    xlim=(0, 1),
    ylim=(0, 1),
    width=300,
    height=300,
    opacity=0.1,
)

Make a map of actual fraction insects


In [ ]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["observed"],
    cmap="reds",
    clim=(0, 0.2),
    size=5,
    opacity=0.75,
)

Make a map of predicted fraction insects


In [ ]:
plot.carto(
    lat=pf["lat"],
    lon=pf["lon"],
    color=pf["predicted"],
    clabel="fraction_insects * mortality",
    cmap="reds",
    clim=(0, 0.2),
    size=2,
    opacity=0.75,
)